# Changing and Rotating Axis

In [7]:
def rotateUniVec(p, e1, e2, e3):
    """returns pPrime
    p is array of vectors in original coordinates of shape (samples, 3)
    e1, e2, and e3 are the new unit vectors in the old axis, each of shape (3)"""
    points = p[:,0].size
    
    x = np.zeros(points)
    y = np.zeros(points)
    z = np.zeros(points)

    for i in np.arange(points):
        x[i] = np.dot( p[i], e1 )
        y[i] = np.dot( p[i], e2 )
        z[i] = np.dot( p[i], e3 ) 
        
    pPrime = np.array([x, y, z]).T
        
    return pPrime
    
def distFromPlane(xP, yP, zP, xC, yC, zC, nx, ny, nz):
    """ For now, normal has to have magnitude 1 """
    return nx*xP + ny*yP + nz*zP - (nx*xC + ny*yC + nz*zC)

In [8]:
# The Okada function only works with strike = 90 degrees. 
# Rotation of points is thus needed

def rotation(strike, x, y, reverse = False): 
    """returns: (xR, yR) 
    
    Provide strike of center fault.
    x and y are location in meters of rotation points.
    Reverse = False means that the x axis is rotated counter clockwise to strike 
    """
    # Make sure strike isn't in an array
    strike = float(strike)
    
    
    # rotates X Y input values so that the faults strike is the new x axis
    xyR = np.zeros((2, x.size))#2d matrix containing all rotated coordinates
    theta = (-strike+ (90 * np.pi / 180) ) #theta is measured from x axis
    
    #-strike doesn't undo the forward reverse! -theta does. 
    if reverse:
        theta = -theta
    
    #rotation is just a standard 2d rotation matrix
    rotation = np.array([[np.cos(theta), np.sin(theta)], 
                            [-np.sin(theta), np.cos(theta)]])
    
    xyR[:] = np.dot( rotation, np.array([x, y]) )#rotate the old x and y coordinates
    
    #unpack the matrix into two 1d arrays
    xR = xyR[0,]
    yR = xyR[1,]
    
    return xR, yR

In [9]:
# Latitudes and Longitudes need to be converted to meters.
# points also need to be centered around some center location

def ChangeAxis(latC, lonC, lat, lon, zone=33, usePyProj=True):
    """returns: (y, x) in meters 
    
    latC and lonC: position of center point. Usually a fault location.
    lat and lon are points to be shifted and converted to meters
    
    usePyProj means use UTM instead of spherical earth approximation.
    """
    
    if usePyProj:       
        proj = pyproj.Proj(proj='utm',zone=zone,ellps='WGS84', eastNorth=True)
        x, y = proj(lon, lat)
        xC, yC = proj(lonC, latC)  
    
    elif not usePyProj:
        xC = np.cos(latC/180.*np.pi)*6371e3*lonC/180.*np.pi
        yC = 111e3*latC
        x =  np.cos(lat /180.*np.pi)*6371e3*lon /180.*np.pi
        y =  111e3*lat
    
    
    x = x - xC
    y = y - yC
    
    return y, x

def xyToLonLat(latC, lonC, x, y, zone = 33, usePyProj = True):
    """
    return (lat, lon)
    
    latC, lonC: floar or nd arrays. Center of cartesian coordinate system. 
    x, y: 1d array with all x and y points. 
    
    Go from x and y back to latitude and longitude knowing what the center 
    Latitude and Longitude of the xy coordinate system is.
    """
    if usePyProj:
        proj = pyproj.Proj(proj='utm',zone=zone,ellps='WGS84', eastNorth=True)
        xC, yC = proj(lonC, latC)
        xNew = xC + x
        yNew = yC + y

        lon, lat = proj(xNew, yNew, inverse=True)
    
    elif not usePyProj:
        # Simply undo formulas in Change Axis
        lat = y/(111e3) + latC

        r = 6371e3 # radius of earth

        lon = 180 / np.pi * ( (x + r * np.cos(latC * np.pi/180) * lonC * np.pi / 180)
                              / (r * np.cos(lat * np.pi / 180)) )
    
    return lat, lon

In [10]:
# Again needed begause the stress tensor is calculated in a modified coordinate system.
# it must be rotated back

def rotationTens(strike, tens, reverse = False):  
    """returns: (tensR) 
    
    rotated so strike is x axis. Accepts strike value, and x and y (centered and in meters).
    Us reverse to indicate direction: rotate so cs is parrallel to strike, or undo that rotation."""
    
    size = tens[:,0,0].size
    theta = (-strike+ (90 * np.pi / 180) )#measured from x axis
    if reverse:
        theta = - theta
    tensR = np.zeros(((size, 3, 3)))#rotated tensors
    
    #standard 3d rotation matrix where there is no vertical rotation
    rotation = np.array([ [np.cos(theta), np.sin(theta), 0],
                        [-np.sin(theta), np.cos(theta), 0],
                        [0, 0, 1] ])
    
    #rotated tensor is R dot tensor dot transpose(R)
    for i in range(size):
        tensR[i] = np.dot( rotation, np.dot(tens[i], rotation.T) )
        
    return tensR

# Access Okada Wrapper

success, u, grad_u = dc3dwrapper(alpha, [X, Y, Z],
    Depth, Dip, [strike distance, strike distance other way], [dip distance, dip distance 
    other way], [strikeslip, dipslip, overlap])<br>
    
alpha = 1/(2*(1-sigma)) <br>
alpha = (lambda + mu) / (lambda + 2 * mu)<br>
xo = 3-vector representing the observation point (x, y, z in the original Okada program)<br>
depth = the depth of the fault origin<br>
dip = the dip-angle of the rectangular dislocation surface<br>
strike_width = the along-strike range of the surface (al1,al2 in the original)<br>
dipLengthHalf = the along-dip range of the surface (aw1, aw2 in the original)<br>
dislocation = 3-vector representing the direction of motion on the surface (DISL1, DISL2, DISL3)   <br> 

In [11]:
# Used to access the Okada wrapper.
# To install okada_wrapper on linux, I used this in terminal: sudo apt-get install gfortran

def dispSum(lambdaLame, shearMod, xP, yP, depthP, depthF, dipF,
            strikeLength, dipLength, strikeSlip, dipSlip, tensile, simBool, centerOrigin=True):
             
    dx = np.zeros(xP.size)
    dy = np.zeros(xP.size)
    dz = np.zeros(xP.size)
    grad = np.zeros((xP.size, 3, 3))
             
    alpha = (lambdaLame + shearMod) / (lambdaLame + 2 * shearMod) # value provided to okada function 
    if centerOrigin is True:
        for i in np.arange(xP.size):
            if simBool[i]:
                __, [dx[i], dy[i], dz[i]], grad[i] = dc3dwrapper(alpha,
                    [xP[i], yP[i], depthP[i]],
                    np.abs(depthF), dipF * 180 / np.pi, 
                    [-.5*strikeLength, .5*strikeLength],
                    [-.5*dipLength, .5*dipLength],
                    [strikeSlip, dipSlip, tensile])
                
    elif centerOrigin == 'bottomCenter':
        for i in np.arange(xP.size):
            if simBool[i]:
                __, [dx[i], dy[i], dz[i]], grad[i] = dc3dwrapper(alpha,
                    [xP[i], yP[i], depthP[i]],
                    np.abs(depthF), dipF * 180 / np.pi, 
                    [-.5*strikeLength, .5*strikeLength], # left range, right range
                    [0, dipLength], # bottom range, top range
                    [strikeSlip, dipSlip, tensile])
                
#     elif not centerOrigin:
#         for i in np.arange(xP.size):
#             if simBool[i]:
#                 __, [dx[i], dy[i], dz[i]], grad[i] = dc3dwrapper(alpha,
#                     [xP[i], yP[i], depthP[i]],
#                     np.abs(depthF), dipF * 180 / np.pi, 
#                     [-.5 * strikeLength, strikeLength],
#                     [-dipLength, 0],
#                     [strikeSlip, dipSlip, tensile])
            
        
    return dx, dy, dz, grad

# Stress and Strain

In [12]:
# Unit vectors of displacement are needed to get normal and shear stress from stress tensor

def faultUnitVectors(s, d, r):  
    """returns (normals, slips) as unit vectors
    
    s, d, r: strike, dip, and rake in radians
    
    Calculate unit vector that describe direction of slip on fault and direction of normal on fault
    These are derived for x=east, y=north, and z=up axis.
    """
    # Facing hanging wall. This means that n dot stress tensor returns stress that
         # the hanging wall exerts onto the footwall. 
    normals = np.array([np.sin(d)*np.cos(s), 
                       -np.sin(d)*np.sin(s),
                       np.cos(d)])
    
    slips = np.array([ -np.cos(s)*np.cos(d)*np.sin(r)+np.sin(s)*np.cos(r),
                      np.sin(s)*np.cos(d)*np.sin(r)+np.cos(s)*np.cos(r),
                     np.sin(d)*np.sin(r)])
    
    return normals.transpose(), slips.transpose()

In [13]:
# Used to calculate the strain tensor from the gradient tensor

def gradToStrainTens(grad):
    '''returns (strainTen). 
    
    give the gradient at all points. 
     
    Uses the standard strain tensor function. '''
    
    strainTen = np.zeros(grad.shape)
    
    #Note, it would be easier to do: strainTen = grad.T+grad)/2
    #But the above would require careful transposing.
    #This gives the same values
    strainTen[:,0,0] = grad[:,0,0]
    strainTen[:,0,1] = .5 * (grad[:,0,1] + grad[:,1,0])
    strainTen[:,0,2] = .5 * (grad[:,0,2] + grad[:,2,0])
    
    strainTen[:,1,0] = .5 * ( grad[:,1,0] + grad[:,0,1] )
    strainTen[:,1,1] = grad[:,1,1]
    strainTen[:,1,2] = .5 * (grad[:,1,2] + grad[:,2,1])
    
    strainTen[:,2,0] = .5 * (grad[:,2,0] + grad[:,0,2])
    strainTen[:,2,1] = .5 * (grad[:,2,1] + grad[:,1,2])
    strainTen[:,2,2] = grad[:,2,2]
    
    return strainTen

The stress tensor is given as

$ \sigma = 2 \mu \epsilon +  \lambda tr\left( \epsilon\right) I$

In [14]:
# Use this to go from gradient to stress tensor.
# Okada wrapper only gives gradient, so this function is needed.

def stressTenFun(grad, mew, lambdaLame):
    """returns (stressTen).
    
    grad is deformation gradient
    mew=shear modulus, lambdalame = other lame constant. """
    
    strainTen = gradToStrainTens(grad)#get strain tensors from gradient
    
    num = strainTen[:, 0, 0].size # number of points
    stressTen = np.zeros(((num, 3, 3)))#initialize
    
    #stressTen = 2 * mew * strainTen + lambdaLame * (trace of strainTen) * identity
    for i in range(num):
        stressTen[i] = 2 * mew * strainTen[i] + lambdaLame * np.trace(strainTen[i]) * np.identity(3)
        
    return stressTen

Coulomb stress is given as:

$CFF = \sigma_{s}+\mu \left( \sigma_{n} - \frac{B}{3}tr \left(\sigma \right)   \right)$

The change in Coulomb stress just replaces the stress values with changes in those stress values

In [15]:
# This finds the change in Coulomb Stress. 
# It also gives some other less important stress values

def coulombStress(dStressTen, u, B, normals=None, slips=None, s=None, d=None, 
                  r=None, regStressTen = np.zeros([1, 3, 3]) ): 
    """return (sigSM, sigNM, dCS, stressTen, sigS)
    
    dStressTen is the change in stress tensor at points
    u is coefficient of friction across a fault
    B is Skemptons coefficient
    normal is the unit vector that is normal to fault planes.
    slips is the unit vector that describes the slip direction of the hanging wall.
    s, d, and r are the strike, dip, and rake.
        Only either the normals and slips are provided, or s, d, and r are provided.
    regStressTen is the regional stress tensor.
        By default, regional stress tensor is zeros and only dCS is returned, not CS
    """

    stressTen = dStressTen + regStressTen
    
    if normals is None:
        normals, slips = faultUnitVectors(s, d, r)
    
    num = int(stressTen.size/9) #1 if for 1 fault, but can handle large arrays to
    tN = np.zeros((num, 3)) #traction projected to normal of future fault planes
    sigN = np.zeros((num, 3)) #normal stress vector
    sigS = np.zeros((num, 3)) #shear stress vector 
    sigNM = np.zeros(num) #sigN in direction of normal
    sigSM = np.zeros(num) #sigS magnitude
    stressTen = stressTen.reshape(num, 3, 3).copy()
    normals = normals.reshape(num, 3).copy()
    slips = slips.reshape(num, 3).copy()
    sigSSlip = np.zeros(num) #shear stress in direction of slip (this is how it's used in coulomb stress equation)
    
    for i in range(num):
        tN[i] = np.dot(stressTen[i], normals[i]) #get traction on fault
        sigNM[i] = np.dot(normals[i], tN[i]) #normal stress scalar, negative means compression... ; normal dot traction
        sigN[i] = sigNM[i] * normals[i] #normal stress = sigNM dot normal unit vector
        sigS[i] = tN[i] - sigN[i] #total shear stress vector accross plane
        sigSM[i] = norm(sigS[i]) #magnitude of shear stress on plane (value is returned and not used in this function)
        sigSSlip[i] = np.dot(sigS[i], slips[i]) #project total shear stress vector into direction of slip
    
    dCS = sigSSlip + u * (sigNM - B / 3 * np.trace(stressTen, axis1 = 1, axis2 = 2)) #calculate change in coulomb stress
    
    return(sigSM, sigNM, dCS, stressTen, sigS)

In [16]:
# If more than one fault or fault patch is simulated onto a point,
# The gradients can't simply be added. This introduces very tiny error with such small gradients.
# Thus, for now, I just sum them. 

def gradientDefSum(gradTotal, gradNew):
    """returns summed gradient.
    
    Give the old gradient and the new value that is being added.
    
    For large gradient, we can't simply sum old and new gradient. """
    #gradTotal = gradTotal + gradNew + gradTotal*gradNew*np.identity(3)
    #gradTotal = gradTotal + gradNew + gradTotal*gradNew
    gradTotal = gradTotal+gradNew
    return gradTotal

In [65]:
def getJ2(stressTen):
    
# Another similar implimentation which will give same results. 
#     s = stressTen
#     J2 =  ( 1/6 * ( (s[0,0]-s[1,1])**2 + (s[1,1] - s[2,2])**2 + (s[2,2]-s[0,0])**2 ) +
#                (s[0,1]**2 + s[1,2]**2 + s[2,0]**2)
#               )
    
    
    points = stressTen.shape[0]
    stressTen = stressTen.reshape(points, 3, 3)
    J2 = np.zeros(points)
    
    for i in range(points):
        J2[i] = .5 * ( np.trace(np.dot(stressTen[i], stressTen[i])) 
                      - 1/3 * np.trace(stressTen[i]) ** 2 )

    return J2

# Selecting Focal Plane

In [19]:
# This is used to sort planes in rupture mechanisms based on each planes similarity to the mainshock

def sortRupture(ST1, ST2, DIP1, DIP2, RK1, RK2, idealAngle):
    """returns (s1, s2, d1, d2, r1, r2) 
    
    ST1 is the strike of the first focal plane, ST2 is the strike of the second.
    Dip and rake follow this logic. 
    
    the returned s1 etc. values have a more similar strike to idealAngle"""
    #chooses the rupture that has strike most similar to idealAngle
    #this is used to consider only the CMT planes striking most similar to the mainshock (which makes some error)
    s1 = np.zeros(ST1.size)
    s2 = s1.copy()
    d1 = s1.copy()
    d2 = s1.copy()
    r1 = s1.copy()
    r2 = s1.copy()
    
    choose1 = np.abs(idealAngle - ST1) < np.abs(idealAngle - ST2)#boolean for choosing first plane
    choose2 = np.abs(idealAngle - ST1) >=np.abs(idealAngle - ST2)#boolean for choosing second plane
    
    s1[choose1] = ST1[choose1]; s1[choose2] = ST2[choose2]
    d1[choose1] = DIP1[choose1]; d1[choose2] = DIP2[choose2]
    r1[choose1] = RK1[choose1]; r1[choose2] = RK2[choose2]
    
    s2[choose1] = ST2[choose1]; s2[choose2] = ST1[choose2]
    d2[choose1] = DIP2[choose1]; d2[choose2] = DIP1[choose2]
    r2[choose1] = RK2[choose1]; r2[choose2] = RK1[choose2]
    
    return s1, s2, d1, d2, r1, r2

My goal here is to determin which of the two focal planes is correct for some of the larger events. While I can simulate deformation onto both of the planes, simulating rupture FROM the wrong plane become quite problematic. 

I haven't found success yet. I plan to using the plotAgainstStrike function.

In [20]:
#distinguish focal planes. Not in use as of June 4th because focalChoices() isn't quite there yet.

def sdrKnown(sw, se, dw, de, rw, re):
    """returns (s, d, r). 
    
    s, d, and r are strike dip and rake.
    w and e signifies that a plane strikes closer or further to the ~west dipping mainshock.
    
    Uses the focalChoices function, where I manually picked the planes."""
    
    s = np.zeros(sw.size)
    d = np.zeros(dw.size)
    r = np.zeros(rw.size)
    
    westDip, westDipLikely, eastDip, eastDipLikely, unknown = focalChoices()
    
    if westDip.size>0:
        s[westDip] = sw[westDip]
        d[westDip] = dw[westDip]
        r[westDip] = rw[westDip]
    
    if eastDip.size>0:
        r[eastDip] = re[eastDip]
        s[eastDip] = se[eastDip]
        d[eastDip] = de[eastDip]
    
    return s, d, r

In [21]:
# Not in use as of June 4th
# The goal here is to decide which of the planes is correct in the larger aftershock.
# My problem was that the indicies don't update automatically

def focalChoices():
    """Returns (westDip, westDipLikely, eastDip, eastDipLikely, unknown)
    
    west means that the plane dips closer to the ~west dipping mainshock. 
        Thus, west is an abbreviation for eastWest
    """
    print('find a way to change from indicies of full vs partial earthquake list')
    westDip = np.array([1428, 76, 242, 775, 294, 211, 217, 13, 63, 6, 115, 7, 844, 875])
    westDipLikely = np.array([1033, 47, 2412])
    eastDip = np.array([])
    eastDipLikely = np.array([88, 89])
    unknown = np.array([233, 2636, 1154, 2138, 2098, 8])
    
    westDip = np.sort(westDip)
    westDipLikely = np.sort(westDipLikely)
    eastDip = np.sort(eastDip)
    eastDipLikely = np.sort(eastDipLikely)
    unknown = np.sort(unknown)
    
    return westDip, westDipLikely, eastDip, eastDipLikely, unknown

In [22]:
# Find the distance from each hypocenter to every other hypocenter

def hypocenterDistances(lat, lon, d):
    """returns (dist). 
    
    give lat, lon, depth (d), and returns an array of distances between all locations
    
    format is dist[fault 1 index, fault 2 index] = distance between fault 1 and 2."""
    
    #We need to convert to meters from lat lon. 
    # First find a basically arbitrary center of coordinate system. 
    latC = np.average(lat) 
    lonC = np.average(lon)
    
    # Now change axis to meters from lat and lon
    y, x = ChangeAxis(latC,lonC,lat, lon) 
    
    dist = np.zeros((lat.size, lat.size))
    
    # dist from fault i to fault n is norm( [xi-xn],[yi-yn],[di-dn] )
    for i in np.arange(lat.size):
        dist[i,:] = norm( np.array([ x-x[i], y-y[i], d-d[i] ]).T, axis=1 )
    #dist is in this format dist[fault 1 index, fault 2 index] = distance between fault 1 and 2
    
    return dist

# Choosing Which Faults Aren't Simulated

In [23]:
# This is to compare the distance between hypocenters to the radius of each rupure

def distVsRad(lat, lon, d, rad):
    """returns (radSumDist, radSourceDist). 
    Basically it's radius/distance. 
    
    d is depth, rad is radius.
    
    Returns two different arrays. 
        radSumDist compares the sum of both radii to a distance. 
        radSourceDist compares just the source quakes radius to a distance.
    
    radSumDist = (rad1+rad2)/dist. radSourceDist = (radSource)/dist"""
    
    # First, find the distances. 
    #Indexes are: dist[source earthquake, secondary earthquake]
    dist = hypocenterDistances(lat, lon, d)
    
    radSource = np.zeros(dist.shape)
    radSecond = np.zeros(dist.shape)
    radSumDist = np.zeros(dist.shape)
    radSourceDist = np.zeros(dist.shape)
    
    #make 2d arrays containing radii. 
    for i in np.arange(lat.size):
        radSource[:, i] = rad
        radSecond[i, :] = rad
    
    #radSum[i,j] is rad[i]+rad[j]
    radSum = radSource + radSecond
    
    # Leave out the case where distance is compared from one hypocenter to itself.
    radSumDist[dist!=0] = radSum[dist!=0]/dist[dist!=0]
    radSourceDist[dist!=0] = radSource[dist!=0]/dist[dist!=0]
    
    return radSumDist, radSourceDist

In [24]:
def getSimBool(fSize, pSize, tF=None, tP=None):
    """
    returns (simBool) 
    
    tF: is t (time) of source fault
    tP: is t (time) of stressed fault
    
    if all simulations are desired, set default argument in sumAll to make all tF<tP
    
    simBool format: simBool[source index, stressed index] = Boolean, do or don't simulate rupture. 
    simBool makes it so that a source fault only stresses future faults.
    """
    simBool = np.ones((fSize, pSize), dtype = 'int')
    
    if not (tP is None): # and (tF is None):
        for i in np.arange(tF.size):
            simBool[i,:] = tP>tF[i]
    
    return simBool

# Combining Other Functions 

In [25]:
# used to count how many mainshocks there are if a nested list of fault parameters is given. 

def sourceNumber(faultList):
    mainshocks = 0
    for i in faultList:
        mainshocks += 1
    return mainshocks

In [26]:
# This function is used to access the other fundamental functions in one go.
# Every time Coulomb Stress or displacement needs to be calculated,
# this function does it. 

def sumAll(shearMod, lambdaLame, coefFrict, coefSkempt,  
            
           latF, lonF, depthF, 
           latP, lonP, depthP, 
            
           dipLength, strikeLength, 
           dipSlip, strikeSlip, tensile,
            
           sF, dF, rF,
           sP=None, dP=None, rP=None,
           
           ttF = None, ttP = None,
            
           strainers=np.array([True]),
           regionalStress=None,
           centerOrigin=True):
    """ 
    rF appears to not be used anymore (08/04/2018)
    
    shearMod, lambdaLame, coefFrict, coefSkempt: Necessary coefficients float.
    latF, lonF, depthF: location of SOURCE FAULT as LISTS nested with arrays
    latP, lonP, depthP: location of stressed points as Arrays
    
    dipLength, strikeLength: LIST of total fault length and width, nested? arrays? 
    dipSlip, strikeSlip, tensile: LIST of dipslip, strikeslip, and tensile displacement as nested? arrays?
        
    sF, dF, rF: Source fault strike dip and rake as ARRAYS
    sP, dP, rP: stressed points strike rake and dip as 1d arrays. 
        Coulomb stress is calculated if these are provided.
        
    ttF, ttP: times that source fault F and stressted point P ruptured. 
    
    strainers: Boolean array to indicate which provided faults provide stress. 
    regionalStress: nx3x3 array. Can provide regional stress at each point, 
        and the function will return total coulomb stress. 
    """
    # How many source quakes and recipient points are there. 
    sourceNum = sourceNumber(latF)
    pointNum = latP.size
    
    # Boolean array to dictate what points recieve stress from what source quakes
    simBool = getSimBool(sourceNum, pointNum, ttF, ttP)
    
    # Initialize the totalled strain related values
    dx =       np.zeros(pointNum)
    dy =       np.zeros(pointNum)
    dz =       np.zeros(pointNum)
    grad = np.zeros((pointNum,3,3))
    
    alpha = (lambdaLame + shearMod) / (lambdaLame + 2 * shearMod) # value provided to okada function
    
    # Start strain loop
    # Loop over all source faults
    for i in np.arange(sourceNum):
        # Loop over all subfaults
        for j in np.arange(dipLength[i].size):
            
            # Convert from lat lon to meters
            yP, xP = ChangeAxis( latF[i][j], lonF[i][j], latP, lonP)#, usePyProj=useUTM )
            
            # Rotate points around center of sub fault
            xPR, yPR = rotation(sF[i][j], xP, yP)
            
            # Find displacement at each point using Okada program
            dxNewR, dyNewR, dzNew, gradNewR = dispSum(lambdaLame, shearMod, xPR,
                yPR, depthP, depthF[i][j], dF[i][j], strikeLength[i][j], 
                dipLength[i][j], strikeSlip[i][j], dipSlip[i][j], tensile[i][j], 
                simBool[i,:], centerOrigin=centerOrigin)
            # Rotate displacement and grad back to neutral coordinate system
            dxNew, dyNew = rotation(sF[i][j], dxNewR, dyNewR, reverse=True)
            gradNew = rotationTens(sF[i][j], gradNewR, reverse=True)
            
            # Sum new displacement to old displacement
            dx+=dxNew
            dy+=dyNew
            dz+=dzNew
#             print('ahoy',gradNew)
            grad = gradientDefSum(grad, gradNew)

    if sP is not None:
        normalsP, slipsP = faultUnitVectors(sP, dP, rP)
        stressTen = stressTenFun(grad, shearMod, lambdaLame)
        
        if regionalStress is not None: 
            stressTen = stressTen + regionalStress
            
        __,__, CS, __, sigSVector = coulombStress(stressTen, coefFrict, coefSkempt, normalsP, slipsP)
    
    elif sP is None:
        CS = None; stressTen = None; slipsP = None; sigSVector = None
         
    return (CS, dx, dy, dz, grad, stressTen, slipsP, sigSVector)

In [3]:
# This function is used to access the other fundamental functions in one go.
# Every time Coulomb Stress or displacement needs to be calculated,
# this function does it. 

def sumAllTD(shearMod, lambdaLame, coefFrict, coefSkempt,  
            
           latF, lonF, depthF, 
           simplicesF,
             
           latP, lonP, depthP, 
             
           dipSlip, strikeSlip, tensile,
            
           sP=None, dP = None, rP = None,
           
           ttF = None, ttP = None,
            
           strainers=np.array([True]),
           regionalStress=None,
           dispOrStress = 2):
    # added: simplices
    # removed: dipLength, strikeLength, sF, dF, rF, centerOrigin
    # removing rotations. now, only one time converting to geog coordinates. center coordinate system only once. 
    """ 
    dispOrStress: 0 is disp, 1 is stress, 2 is both
    Can only provide one main fault model now, but it should still be in a list (03/30/2019)
    
    shearMod, lambdaLame, coefFrict, coefSkempt: Necessary coefficients float.
    latF, lonF, depthF: location of SOURCE FAULT as LISTS nested with arrays
    simplices define the 3 indices that define a triangle, for every triangle. 
        shape is (number of triangles, 3 verticy indices for each triangle)
    latP, lonP, depthP: location of stressed points as Arrays
    
    dipSlip, strikeSlip, tensile: LIST of dipslip, strikeslip, and tensile displacement as nested? arrays?

            
    sP, dP, rP: stressed points strike rake and dip as 1d arrays. 
        Coulomb stress is calculated if these are provided.
        
    ttF, ttP: times that source fault F and stressted point P ruptured. 
    
    strainers: Boolean array to indicate which provided faults provide stress. 
    regionalStress: nx3x3 array. Can provide regional stress at each point, 
        and the function will return total coulomb stress. 
    """
    # How many source quakes and recipient points are there. 
    sourceNum = 1
    pointNum = latP.size
    triangleNum = simplicesF[0].shape[0]
    
    # Boolean array to dictate what points recieve stress from what source quakes
#     simBool = getSimBool(sourceNum, pointNum, ttF, ttP)

    Displ = np.zeros((pointNum, 3), dtype = np.float64)
    Stress = np.zeros((pointNum, 6), dtype = np.float64)
    Strain = np.zeros((pointNum, 6), dtype = np.float64)
    DisplTemp = np.zeros(3, dtype = np.float64)
    StressTemp = np.zeros(6, dtype = np.float64)
    StrainTemp = np.zeros(6, dtype = np.float64)
    
    poisson = lambdaLame / (2 * (lambdaLame + shearMod) )

    
    # Start strain loop
    # Loop over all source faults. well... there is only one source fault...
    for i in np.arange(sourceNum):
        # Loop over all subfaults
        latFAv = np.average(latF)
        lonFAv = np.average(lonF)
        yP, xP = ChangeAxis( latFAv, lonFAv, latP, lonP)
        yF, xF = ChangeAxis( latFAv, lonFAv, latF[0], lonF[0])
        xF = [xF]
        yF = [yF]

        for j in np.arange(triangleNum):
            for k in np.arange(pointNum):
                
                #verify that this strategy works
                triInds = simplicesF[i][j]
                simpx = xF    [i][triInds]
                simpy = yF    [i][triInds]
                simpz = depthF[i][triInds]
                p123 = np.array([simpx, simpy, simpz])
                P1 = p123[:, 0]
                P2 = p123[:, 1]
                P3 = p123[:, 2]
                
                ss = np.average(strikeSlip[i][triInds]) 
                ds = np.average(dipSlip   [i][triInds]) 
                ts = np.average(tensile   [i][triInds])                   
                
                if (dispOrStress == 0) or (dispOrStress == 2):
                    TDDispl.py_DispInHalfSpace(
                        DisplTemp, 
                        xP[k], yP[k], depthP[k], 
                        P1.copy(), P2.copy(), P3.copy(),
                        ss, ds, ts,
                        poisson                )
                    Displ[k] += DisplTemp
                    
                if (dispOrStress == 1) or (dispOrStress == 2):
                    TDStrain.py_StrainInHalfSpace(
                        StressTemp, StrainTemp,
                        xP[k], yP[k], depthP[k], 
                        P1.copy(), P2.copy(), P3.copy(),
                        ss, ds, ts,
                        shearMod, lambdaLame)
                    Stress[k] += StressTemp
                    Strain[k] += StrainTemp
                
            
    dx = Displ[:, 0]
    dy = Displ[:, 1]
    dz = Displ[:, 2]
        
    stressTen = np.zeros((pointNum, 3, 3))
    strainTen = np.zeros((pointNum, 3, 3))
    
    # note that the order here was misdocumented in the original C code. It may change in the future. 
    # I fixed the order on 01/04/2019
    stressTen[:, 0, 0] =                      Stress[:, 0]
    stressTen[:, 1, 1] =                      Stress[:, 3]
    stressTen[:, 2, 2] =                      Stress[:, 5]
    stressTen[:, 0, 1] = stressTen[:, 1, 0] = Stress[:, 1]
    stressTen[:, 0, 2] = stressTen[:, 2, 0] = Stress[:, 2]
    stressTen[:, 1, 2] = stressTen[:, 2, 1] = Stress[:, 4]
    
    strainTen[:, 0, 0] =                      Strain[:, 0]
    strainTen[:, 1, 1] =                      Strain[:, 3]
    strainTen[:, 2, 2] =                      Strain[:, 5]
    strainTen[:, 0, 1] = strainTen[:, 1, 0] = Strain[:, 1]
    strainTen[:, 0, 2] = strainTen[:, 2, 0] = Strain[:, 2]
    strainTen[:, 1, 2] = strainTen[:, 2, 1] = Strain[:, 4]
        
    return (np.nan, dx, dy, dz, strainTen, stressTen, np.nan, np.nan)

# Inversion

In [27]:
def packMats(packedVals = None, stSl=None, dpSl=None):
    """returns packed if stSl is provided.
    returns stSl and dpSl if packed is provided. 
    
    packedVals is the 1d combined array.
    stSl and dpSl are 2d matricies described inhomogenous strike slip and dip slip
    """
    if stSl is not None:
        packed = np.array([stSl, dpSl]).reshape(-1)
        return packed
    if packedVals is not None:
        slips = packedVals.reshape(-1)
        #stSl is first half os slips, dpSl is second half
        stSl = np.array([slips[:int(slips.size/2)]])
        dpSl = np.array([slips[int(slips.size/2):]])
        return stSl, dpSl

In [28]:
# Show us the rms of the error in x, y, and z. 

def rmsResiduals(dxO, dyO, dzO, dxM, dyM, dzM):
    """
    returns rms residuals for modeled and observed displacements. returns in x, y, z order
    give observed (O) and modelled (M) values and find the error between them. 
    """
    o = np.array([dxO, dyO, dzO])
    m = np.array([dxM, dyM, dzM])
    rms = np.zeros(3)
    
    #do rms operation.
    squared = (o-m)**2
    for i in np.arange(3):
        rms[i] = np.sqrt(np.average(squared[i]))
        
    return rms[0], rms[1], rms[2]

In [29]:
def cov_inv(edx, edy, edz, minimum = 1e-20, returnCov=False, returnIdentity=False):
    """
    returns (covInv): inverse of covariance matrix as cov[i,i]=std**2 and cov[i,j]=0. 
    
    All edx comes before edy which comes before edz in the matrix."""
    if returnIdentity:
        return np.identity(edx.size+edy.size+edz.size) 
    
    xyz = np.array([edx, edy, edz]).reshape(-1)
    xyz[xyz<minimum]=minimum # if any value is 0, the determinant becomes 0 and we can't invert
    cov = np.outer(xyz, xyz) * np.identity(xyz.size)
    
    if returnCov:
        return cov
    
    covInv = inv(cov)

    return covInv

In [30]:
# Used in heterogenous slip inversion
#Laplacian in 2d with finite difference approximtaion.
#Quantifies smoothness for heterogenous slip distribution. 

def zeroPerim(mat, forward = True):
    """returns (mat):
    
    adds zeros to the edges of the ruptured part of fault so that displacement tapers to zero.
    Valid because we assume displacement is zero outside of fault.
    if forward==False, removes these extra zeros.
    """
    if forward:
        matNew = np.zeros((mat.shape[0]+2, mat.shape[1]+2))
        matNew[1:-1, 1:-1] = mat    
        return matNew
    if not forward:
        return mat[1:-1,1:-1]

def d_dx(mat, colWidth):
    """Derivative of function (mat) in x direction. (f(x+h)-f(x-h))/2h"""
    der = np.zeros(mat.shape)
    der[:, 1:-1]=(mat[:,2:]-mat[:, 0:-2])/(2*colWidth)
    der[:, -1]= (0 - mat[:,-2])/(2*colWidth)
    der[:, 0] = (mat[:, 1] - 0 )/(2*colWidth)
    return der

def d_dy(mat, rowHeight):
    """Derivative of function in y direction. (f(y+h)-f(y-h))/2h"""
    der = np.zeros(mat.shape)
    der[1:-1,:] = (mat[2:,:]-mat[0:-2])#/(2*rowHeight)
    der[-1, :] = (0 - mat[-2, :])/(2*rowHeight)
    der[0, :] = (mat[1, :] - 0)/(2*rowHeight)
    return der

def lapTotal(ss, ds, colWidth, rowHeight):
    """returns (lapT): laplacian
    
    Laplacian function. assumes that there is 0 displacement outside of fault. Thus, displacement tapers to 0. 
    
    ss and ds are strike slip and dip slip.T"""
    ss = zeroPerim(ss)
    ds = zeroPerim(ds)
    lapX = d_dx(d_dx(ss, colWidth),colWidth)
    lapY = d_dy(d_dy(ds, rowHeight),rowHeight)
    lapT = lapX+lapY
    lapT = zeroPerim(lapT, forward=False)
    return lapT

In [31]:
# Used for uniform slip inversion

def erFinderHom(firstGuess, lonO, latO, dxO, dyO, duO, depthO, strikeRects, verticalRects,
              shearMod, lambdaLame, coefFrict, coefSkempt, 
            edx=None, edy=None, edu=None, returndxyz = False):
    """returns (WRSS): weighted residual sum of squares. 
    
    First guess: (lonF, latF, depth, s, d, r, slip, 
           strikeLength, dipLength)
           
           it is a 1d list containing all values that are being inverted.
           
    The O suffix means observed, for instance, observed dx at GPS station. 
    """
    
    # unpack the starting point values, careful to keep them each as a 1d arrays.  
    (lonF, latF, depth, s, d, r, slip, 
           strikeLength, dipLength) = np.array([firstGuess]).T
    
    # Get strikeSlip and dipSlip from strike and rake
    ss = np.cos(r) * slip
    ds = np.sin(r) * slip

    # Put parameters in list form.
    (s, d, r, strikeLength, dipLength, depth, lonF, latF, ds, ss, tensile
        )=listFaults(s, d, r, ds, ss, strikeLength, dipLength, 
            depth, lonF, latF, np.array([strikeRects]).reshape(-1),
            np.array([verticalRects]).reshape(-1))
    
    
    (__, dx, dy, dz, __, __, __, __
        )=sumAll(shearMod, lambdaLame, coefFrict, coefSkempt, latF, lonF, 
            depth, latO, lonO, depthO, dipLength, strikeLength,
            ds, ss, tensile, s, d, r)

    # if we aren't inverting, return the errors instead. 
    if returndxyz:
        return dx, dy, dz
    
    if edx is not None:
        #Make error residual array, dot it to covariance inverse, and dot it to transpose residual array.
        dXYZ = np.array([ [dxO-dx], [dyO-dy], [duO-dz] ]).reshape(-1)
        cov = cov_inv(edx, edy, edu)
        WRSS = np.dot( dXYZ, np.dot(cov, dXYZ.T) )
        #TODO: can get rid of a dot and do normal multiplication to separate the errors?
        
    return WRSS

In [32]:
# Used for uniform slip inversion

def erFinderHet(firstGuess, 
                
              b, s, d, lonF, latF, depthF, strikeLength, dipLength,
              
              lonO, latO, dxO, dyO, duO, depthO, strikeRects, verticalRects, 
              shearMod, lambdaLame, coefFrict, coefSkempt, 
              edx=np.array([False]), edy=np.array([False]), edu=np.array([False]), 
              returndxyz = False):
    """returns (WRSS): weighted residual sum of squares. 
    
    First guess: maybe... (lonF, latF, depth, s, d, r, slip, 
           strikeLength, dipLength)
           
           it is a 1d list containing all values that are being inverted.
           
    The O suffice means observed, for instance, observed dx at GPS station. 
    """
    #Just a dummy variable because I use strike slip and dip slip instead. 
    r = s
    
    #unpack strikeSlip and dipSlip arrays
    ss, ds = packMats(packedVals=firstGuess)

    # Put parameters in list form.
    (s, d, __, strikeLength, dipLength, depthF, lonF, latF, __, __, tensile
        )=listFaults(s, d, r, r, r, strikeLength, dipLength, 
            depthF, lonF, latF, np.array([strikeRects]).reshape(-1),
            np.array([verticalRects]).reshape(-1))    
    
    (__, dx, dy, dz, __, __, __, __
        )=sumAll(shearMod, lambdaLame, coefFrict, coefSkempt, latF, lonF, 
            depthF, latO, lonO, depthO, dipLength, strikeLength,
            ds, ss, tensile, s, d, r)
    
    # if we aren't inverting, return the errors instead. 
    if returndxyz:
        return dx, dy, dz
    
    if edx is not None:
        #Make error residual array, dot it to covariance inverse, and dot it to transpose residual array.
        dXYZ = np.array([ [dxO-dx], [dyO-dy], [duO-dz] ]).reshape(-1)
        cov = cov_inv(edx, edy, edu)
        WRSS = np.dot( dXYZ, np.dot(cov, dXYZ.T) )
        #TODO: can get rid of a dot and do normal multiplication to separate the errors.
        
    return WRSS

# Move between different earthquake values (mw, mo, radius, etc.)

In [1]:
def mLtomW(mL):
    # From Munafò et al., 2016 - BSSA
    toBig = mL>4
    if toBig.sum() > 0:
        print(toBig.sum(), ' # larger than mL 4. Calculation ran anyway.: mL = ', mL[toBig])
    mW = mL*2/3 + 1.15
        
    return mW

def mWtomO(mW):

    # old implementation commented out
    #     mO = 10 ** (1.5 * mW + 16.1)
    #     mO = mO * 1e-5 * 1e-2 # 1d = 1e-5N   1m = 1e-2cm

    mO = 10 ** (3/2 * (mW + 6.07) ) # Hanks and Kanamori (1979)
    return mO

def mOtomW(mO):
    mW = 2/3 * np.log10(mO) - 6.07
    return mW

def mOtoRad(mO, stressDrop, P):
    """P is Poisson"""
    # Cambiotti described this
    rad = 1/2 * (
            3/2 * 
           (2-P)/(1-P) * 
           (mO / stressDrop)
        
          ) ** (1/3) 
    return rad

def radTomO(rad, stressDrop, P):
    mO = (rad*2) ** 3 * 2/3 * (1-P)/(2-P) * stressDrop
    return mO
    

def slipConstant(shearMod, area, moment):
    # From moment = shearMod * area * slip
    slipConstant = moment / (shearMod * area)
    return slipConstant

def ellipSlipMax(slipAv, area, yMax, xMax):
    # I wrote the solution to this. It's found by suggesting that moment is conserved
    # moving from an constant slip to elliptical slip. 
    sMax = 3/(2 * np.pi) * (slipAv * area) / (yMax * xMax)
    return sMax